In [1]:
import clsDatabase as db
import ClsModel as mo
#pip install pandas 필요
import numpy as np
#pip install numpy

#테스트용
import random

#delete 필요
#pip install scikit-learn, scipy 필요

In [2]:
#Database에서 자료 가져오기
query = 'SELECT candle_date_time_utc, candle_date_time_kst, opening_price, high_price, low_price, '
query +=      ' trade_price, timestamp, candle_acc_trade_price, candle_acc_trade_volume, unit,'
query +=      ' gap, [check] '
query +=      ' FROM TradingTest'

#table type : List
table = db.SelectQuery(query)

db.ConnectionClose()

In [3]:
trainRate = 0.8 #학습용 데이터 80%, 테스트용 20%
resultColName = 'check'

trainLearningData, trainResultData, testLearningData, testResultData = mo.PreTrainData(table, 'check', 0.8)
model = mo.CreateModel(trainLearningData)
mo.TrainModel(model, trainLearningData, trainResultData, testLearningData, testResultData)

model.summary()

Epoch 1/3
5/5 [==============================] - 1s 43ms/step - loss: 0.0000e+00 - accuracy: 0.4875 - val_loss: 0.0000e+00 - val_accuracy: 0.5750
Epoch 2/3
5/5 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.4875 - val_loss: 0.0000e+00 - val_accuracy: 0.5750
Epoch 3/3
5/5 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.4875 - val_loss: 0.0000e+00 - val_accuracy: 0.5750
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 11)]              0         
                                                                 
 Dense1 (Dense)              (None, 64)                768       
                                                                 
 Dense2 (Dense)              (None, 256)               16640     
                                                                 
 Dense3 (Dense)      

In [4]:
print(trainLearningData.shape)
randRowIdx = random.randrange(0, testLearningData.shape[0])

#InputData 전처리
testInputAry = testLearningData[randRowIdx:randRowIdx+1, :]
testInputData = np.asarray(testInputAry).astype(np.float32)

print(testInputData.shape)
print(testInputData)

(160, 11)
(1, 11)
[[ 2.0211211e+11  2.0211211e+11  6.0525000e+07  6.0536000e+07
   6.0324000e+07  6.0358000e+07  1.6392313e+12  2.1730094e+09
   3.5980042e+01  1.5000000e+01 -1.5400000e+05]]


In [6]:
result = model.predict(testInputData)
maxcount = 0
while int(result) == 1 :
    result = model.predict(testInputData)
    maxcount = maxcount +1
    if maxcount > 1000: break
print(int(result))

1


In [ ]:
# del query
# del value, rows, table
# del trainRate, resultFieldname, data, trainSize, trainData, testData
# del Input, Dense1, Dense2, Dense3, Dense4, Output